In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [3]:
import fitz  # PyMuPDF
import os

# PDF path (can be anywhere)
file_path = r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf"
pdf = fitz.open(file_path)

# Current working directory instead of __file__
project_root = os.getcwd()
output_root = os.path.join(project_root, "output_images")

# Extract PDF name (without extension) for subfolder
pdf_name = os.path.splitext(os.path.basename(file_path))[0]
output_dir = os.path.join(output_root, pdf_name)
os.makedirs(output_dir, exist_ok=True)

# Convert each page to image
for page_number in range(len(pdf)):
    page = pdf[page_number]
    pix = page.get_pixmap(dpi=300)
    output_path = os.path.join(output_dir, f"page_{page_number+1}.png")
    pix.save(output_path)
    print(f"Saved {output_path}")


Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_1.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_2.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_3.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_4.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_5.png
Saved C:\workspace\artelligence\playground\agents\output_images\upload_nic_2018_5c94b4f64e25f\page_6.png


In [6]:
import os
from google import genai

client = genai.Client()

# Folder where images are stored (example: "output_images/upload_nic_2018_5c94b4f64e25f")
folder_path = r"output_images\upload_nic_2018_5c94b4f64e25f"

uploaded_files = []

# Loop through all image files in folder
for filename in sorted(os.listdir(folder_path)):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):  # only images
        file_path = os.path.join(folder_path, filename)
        file = client.files.upload(file=file_path)
        uploaded_files.append(file)
        print(f"✅ Uploaded {filename}, URI: {file.uri}")

print(f"\nTotal uploaded: {len(uploaded_files)} files")


✅ Uploaded page_1.png, URI: https://generativelanguage.googleapis.com/v1beta/files/m14ek7vxsz4y
✅ Uploaded page_2.png, URI: https://generativelanguage.googleapis.com/v1beta/files/ek7klgsf3dgv
✅ Uploaded page_3.png, URI: https://generativelanguage.googleapis.com/v1beta/files/fis5jzol7gwf
✅ Uploaded page_4.png, URI: https://generativelanguage.googleapis.com/v1beta/files/0yueu42a7he1
✅ Uploaded page_5.png, URI: https://generativelanguage.googleapis.com/v1beta/files/5ofwtbd9290y
✅ Uploaded page_6.png, URI: https://generativelanguage.googleapis.com/v1beta/files/m168l62lyamt

Total uploaded: 6 files


In [7]:
print(uploaded_files)

[File(
  create_time=datetime.datetime(2025, 10, 2, 16, 28, 54, 472154, tzinfo=TzInfo(UTC)),
  expiration_time=datetime.datetime(2025, 10, 4, 16, 28, 53, 948965, tzinfo=TzInfo(UTC)),
  mime_type='image/png',
  name='files/m14ek7vxsz4y',
  sha256_hash='NzZmMzVhZWNhMDhmZGJjMTM4OGFhOTY5OTA0OTMyY2Y1NWEwNDQwMTI4NzAwNTFlMzI5MjM5Y2U5MDIwM2RhMA==',
  size_bytes=6952100,
  source=<FileSource.UPLOADED: 'UPLOADED'>,
  state=<FileState.ACTIVE: 'ACTIVE'>,
  update_time=datetime.datetime(2025, 10, 2, 16, 28, 54, 472154, tzinfo=TzInfo(UTC)),
  uri='https://generativelanguage.googleapis.com/v1beta/files/m14ek7vxsz4y'
), File(
  create_time=datetime.datetime(2025, 10, 2, 16, 29, 0, 935388, tzinfo=TzInfo(UTC)),
  expiration_time=datetime.datetime(2025, 10, 4, 16, 29, 0, 301560, tzinfo=TzInfo(UTC)),
  mime_type='image/png',
  name='files/ek7klgsf3dgv',
  sha256_hash='MDMzZTM3N2Y0MWEyODI0Yjc1NTQ1NzFjYTg5NzdjMDQyYzNhMTQ2ZTU4MGVmMzNmYWIyZGJmNzM2NzNlZDQyMg==',
  size_bytes=8487540,
  source=<FileSource.UPLOA

In [3]:
from google import genai

client = genai.Client()

# Directly pass file URIs to the request
file_uris = [
    "https://generativelanguage.googleapis.com/v1beta/files/m14ek7vxsz4y",
    "https://generativelanguage.googleapis.com/v1beta/files/ek7klgsf3dgv",
    "https://generativelanguage.googleapis.com/v1beta/files/fis5jzol7gwf",
    "https://generativelanguage.googleapis.com/v1beta/files/0yueu42a7he1",
    "https://generativelanguage.googleapis.com/v1beta/files/5ofwtbd9290y",
    "https://generativelanguage.googleapis.com/v1beta/files/m168l62lyamt",
]

# Build contents: wrap URIs properly
contents = [{"file_data": {"file_uri": uri}} for uri in file_uris]

# Add your instruction at the end
contents.append(
    """These 6 images are pages of the same inspection document. 
    Extract the following information:
    1. Document type (Gas_safety, EPC, or EIRC)
    2. Address of the inspection
    3. Inspection status (Passed or Failed)

    Return the result strictly as JSON with keys: 
    { "document_type": "", "address": "", "inspection_status": "" }
    """
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
)

print("✅ Document Type:", response.text)


✅ Document Type: ```json
{
  "document_type": "EIRC",
  "address": "42 CHESTERFIELD COURT FLAT 1",
  "inspection_status": "Failed"
}
```


In [15]:
  from google import genai
  from google.genai import types

  with open('page_1.png', 'rb') as f:
      image_bytes = f.read()

  client = genai.Client()
  response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
      types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/png',
      ),
      'Caption this image.'
    ]
  )

  print(response.text)

This is an "Electrical Installation Condition Report" for a landlord safety inspection, dated August 28, 2018.

**Key details from the report include:**

*   **Report No.:** DPN7/0393125
*   **Contractor's Reference Number:** CRN/007
*   **Client:** David Harouni, located at 13A West Hampstead Mews, NW6 3BB.
*   **Installation Location (Occupier):** David Harouni, 42 Chesterfield Court Flat 1, NW11 8AU.
*   **Type of Installation:** Ticked as "Highway Installation" (though the addresses are residential).
*   **Purpose of the Report:** Landlord Safety Inspection.
*   **Date of Inspection:** 28-8-2018.
*   **Estimated Age of Installation:** 20 years, with evidence of 10-year-old alterations.
*   **Records:** Unknown previous inspection date, no previous certificate/report number, no records available.
*   **Extent of Inspection:** 60% of accessories removed for visual inspection, all final circuits tested.
*   **Operational Limitations:** Insulation resistance between live conductors not

In [14]:
# without structure output
from google import genai
from google.genai import types
import pathlib

client = genai.Client()

# Retrieve and encode the PDF byte
filepath = pathlib.Path(r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf")

response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      """ 
      Extract the following information: Document type (Gas_safety, EPC, or EIRC),Address of the inspection,Inspection status (Passed or Failed)
    """])
print(response.text)

- **Document type:** EIRC
- **Address of the inspection:** 42 CHESTERFIELD COURT FLAT 1, NNII SAU
- **Inspection status:** Failed


In [11]:
# with structured output
from google import genai
from google.genai import types
import pathlib
from pydantic import BaseModel

class Recipe(BaseModel):
    doc_type: str
    address: str
    inspection:bool

client = genai.Client()

filepath = pathlib.Path(r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf")

response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      """ 
      Extract the following information:
    1. Document type (Gas_safety, EPC, or EIRC)
    2. Address of the inspection
    3. Inspection status (Passed or Failed)"""],
    config={
        "response_mime_type": "application/json",
        "response_schema": Recipe,
    }
)

print(response)
structured_result: Recipe = response.parsed


sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text='{"doc_type": "EIRC", "address": "42 CHESTERFIELD COURT FLAT 1 NNII SAU", "inspection": false}'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='u0vjaKm8OK2M1e8PruCxmAk' usage_metadata=GenerateContentResponseUsageMetadata(
  cache_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=44
    ),
    ModalityTokenCount(
      modality=<MediaModality.DOCUMENT: 'DOCUMENT'>,
      token_count=1544
    ),
  ],
  cached_content_token_count=1588,
  candidates_token_count=32,
  prompt_token_count=1593,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=45
    ),
    ModalityTokenCount(
      modality=<MediaModality.DOCUMEN

In [12]:
structured_result.doc_type

'EIRC'

In [13]:
structured_result.inspection

False

In [35]:
# automatic function calling with structured output
from google import genai
from google.genai import types 
from pydantic import BaseModel
import pathlib

class structure(BaseModel):
    doc_type:str
    address:str
    inspection:bool


client = genai.Client()

# function to extract info from the pdf
def extract_info(pdf_path:str)->structure:
    """Extract doc_type, address, and inspection status from a PDF"""
    filepath = pathlib.Path(pdf_path)
    print(pdf_path)

    response = client.models.generate_content( model = "gemini-2.5-flash",
            contents = [ types.Part.from_bytes(data = filepath.read_bytes(),mime_type = 'application/pdf',),
                """Extract the following information:
                1. Document type (Gas_safety or  EPC, or EIRC)
                2. Address of the inspection
                3. Inspection status - True (Passed(satisfactory)) / False (Failed(unsatisfactory))"""],
            config = {
                    "response_mime_type":"application/json",
                    "response_schema":structure,
                        }
                                             )
    return response.parsed 

extract_info_decl = types.FunctionDeclaration.from_callable(callable=extract_info, client = client)

tools = types.Tool(function_declarations=[extract_info_decl])

config = types.GenerateContentConfig(tools = [tools],
                                     #automatic_function_calling = True,
                                     thinking_config = types.ThinkingConfig(include_thoughts=True)
                                    )
path = r"C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf"



response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents = f"extract information from this PDF{path}",
    config = config
)
result = None
for part in response.candidates[0].content.parts:
    if part.function_call:
        func_name = part.function_call.name
        func_args = part.function_call.args
        if func_name == "extract_info":
            result: structure = extract_info(**func_args)
            print(result.doc_type, result.address, result.inspection)

        

C:\Users\Raghul mani M\Downloads\compliance_docs\compliance_docs\upload_nic_2018_5c94b4f64e25f.pdf
EIRC 42 CHESTERFIELD COURT FLAT 1 NNII SAU False


In [36]:

print(result.doc_type)
print(result.address)
print(result.inspection)

EIRC
42 CHESTERFIELD COURT FLAT 1 NNII SAU
False


### **Task 2: “Financial Statement Summarizer & Risk Evaluator”**

**Goal:**
Write a function that reads one or more company financial PDFs (like balance sheets or income statements) and outputs structured analytics.

**Output:**

```json
{
  "company_name": "ABC Ltd",
  "fiscal_year": "2024-25",
  "revenue": "₹5.2 Cr",
  "net_profit": "₹80 Lakh",
  "risk_level": "High" | "Medium" | "Low",
  "reasoning": "Net profit margin decreased by 12% due to increased operating costs."
}
```



In [2]:
from google import genai
from google.genai import types
import pathlib
from pydantic import BaseModel
from enum import Enum

class RiskLevel(str, Enum):
    LOW = "Low"
    MEDIUM = "Medium"
    HIGH = "High"
    CRITICAL = "Critical"

class operation(BaseModel):
    company_name: str
    fiscal_year: str
    revenue: str
    net_profit: str
    risk_level:  RiskLevel
    reasoning:str
    

"""
operation = types.Schema(
    type="object",
    properties={
        "company_name": { "type": "string"},
        "fiscal_year": { "type": "string"},
        "net_profit": { "type": "string" },
        "risk_level": { "type": "string","enum": ["High", "Medium", "Low"]},
        "reasoning":{"type":"string"}
    },
)
"""    


client = genai.Client()
filepath = pathlib.Path(r"C:\Users\Raghul mani M\Downloads\infosys-finstatement.pdf")

prompt = """extract the following information from the document
            1. company name 2. fiscal_year 3. revenue 4. net_profit 5. risk_level 6. reasoning """
response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        # input as pdf
        mime_type='application/pdf',
      ),
      prompt],
    
    config={
        "response_schema": list[operation],
        "response_mime_type": "application/json"
    }
    
    )

"""
object_1: operation = response.parsed[0]

print(object_1.company_name)
print(object_1.fiscal_year)
print(object_1.revenue)
print(object_1.net_profit)
print(object_1.risk_level)
print(object_1.reasoning)
"""
print('------------------------------------------')

for item in response.parsed:
    print(item.company_name, item.fiscal_year, item.revenue, item.net_profit, item.risk_level,item.reasoning)
    print()

------------------------------------------
INFOSYS LIMITED 2014 46,917 crore 10,194 crore RiskLevel.LOW Infosys Limited demonstrated strong revenue growth and healthy net profit growth from fiscal year 2013 to 2014. Despite incurring a one-time litigation settlement charge of 
r219 crore in 2014, the company maintained healthy profitability, indicating financial resilience and strong operational performance.

INFOSYS LIMITED 2013 38,980 crore 9,116 crore RiskLevel.LOW Infosys Limited reported healthy revenue and net profit in fiscal year 2013, providing a strong baseline for the subsequent year's growth. No significant adverse financial indicators were noted for this period based on available information.



### **Task 3: “Real Estate Legal Document Validator”**

**Goal:**
Analyze legal property PDFs (sale deed, encumbrance certificate, tax receipt) and check if the documents are consistent.

**Output:**

```json
{
  "property_id": "TN/CHN/12345",
  "owner_name": "R. Kumar",
  "address": "No. 45, Gandhi Street, Chennai",
  "all_documents_consistent": true,
  "issues": ["Owner name mismatch between EC and Deed"]
}
```




In [3]:
# in gemini, in file api is not working with is live connect, other than normal generate content file api is working fine,

from google import genai
from google.genai import types
from google.genai.types import Modality
import asyncio

client = genai.Client()
file = client.files.upload(file=r"C:\Users\Raghul mani M\Downloads\AI_Rover.pdf") 
print("Uploaded file URI:", file.uri)
print("Mime type:", file.mime_type)
async def main():
    async with client.aio.live.connect(
        model="gemini-live-2.5-flash-preview",
        config=types.LiveConnectConfig(
            response_modalities=[Modality.TEXT]
        )
    ) as session:
        turns = [
            types.Content(
                role="user",
                parts=[
                    types.Part.from_uri(file_uri=file.uri, mime_type=file.mime_type)
                ]
            )
        ]
        await session.send_client_content(turns=turns)


    
        async for message in session.receive():
            print(message.text)
            break  

await main()


Uploaded file URI: https://generativelanguage.googleapis.com/v1beta/files/v9v05gdg4ajt
Mime type: application/pdf


ConnectionClosedError: received 1007 (invalid frame payload data) Request contains an invalid argument.; then sent 1007 (invalid frame payload data) Request contains an invalid argument.

In [1]:
from google import genai
from google.genai import types
from google.genai.types import Modality
import json
import asyncio
from dotenv import load_dotenv
import os
from pydantic import BaseModel
from typing import List, Optional

class ValidationResult(BaseModel):
    property_id: str
    owner_name: str
    address: str
    all_documents_consistent: bool
    issues: Optional[List[str]]



load_dotenv()  
api_key = os.getenv("GENAI_API_KEY")
client = genai.Client(api_key=api_key)



uploaded_file_uris = []

def validate_document_1(file_path: str, prompt: str) -> ValidationResult:
   
    uploaded = client.files.upload(file=file_path)
    uploaded_file_uris.append(uploaded.uri)

    contents = [
        types.Content(
            role="user",
            parts=[types.Part(file_data=types.FileData(file_uri=uri))]
        )
        for uri in uploaded_file_uris
    ]
    contents.append(
        types.Content(
            role="user",
            parts=[types.Part(text=prompt)]
        )
    )

    result = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents,
        config={
            "response_mime_type": "application/json",
            "response_schema": ValidationResult,
        }
    )
    return result.parsed

async def main():
    print("Paste JSON input like: {\"file\": \"path/to/file.pdf\", \"prompt\": \"Analyze this\"}")
    print('Type "exit" to quit.')
    while True:
        user_text = input("Enter JSON input or exit: ").strip()
        if user_text.lower() == "exit":
            print("Exiting...")
            break
        try:
            data = json.loads(user_text)
            file_path = data.get("file")
            prompt = data.get("prompt")
            if not file_path or not prompt:
                print(" Missing 'file' or 'prompt' in input.")
                continue
            result = await asyncio.to_thread(validate_document_1, file_path, prompt)
            print("Property ID:", result.property_id)
            print("Owner Name:", result.owner_name)
            print("Address:", result.address)
            print("All Documents Consistent:", result.all_documents_consistent)
            print("Issues:", result.issues)

            print("Response:\n", result)
        except Exception as e:
            print(f" Error parsing input or processing: {e}")


await main()


Paste JSON input like: {"file": "path/to/file.pdf", "prompt": "Analyze this"}
Type "exit" to quit.


Enter JSON input or exit:   {   "file": "C:/Users/Raghul mani M/Downloads/house-tax-receipt.pdf",   "prompt": " analyse and extract information(extract property id, owners name, address) from the given  document  "  }


Property ID: 70/I
Owner Name: செல்வம் sl0 கணேசன்
Address: 70/I, கட்டும அகினைநகர், கணியம்பாடி
All Documents Consistent: True
Issues: None
Response:
 property_id='70/I' owner_name='செல்வம் sl0 கணேசன்' address='70/I, கட்டும அகினைநகர், கணியம்பாடி' all_documents_consistent=True issues=None


Enter JSON input or exit:  {   "file": "C:/Users/Raghul mani M/Downloads/encumbrance-certificate-sample.pdf",   "prompt": " analyse and extract information(extract property id, owners name, address) from current document and all documents consistent(True/False) and issues('issues': ['Owner name mismatch between EC and Deed']) from the given and previous documents"  }


Property ID: 70/I
Owner Name: செல்வம் sl0 கணேசன்
Address: Door No. 70/I, Kattuma Akinainagar, Kaniyambadi Panchayat
All Documents Consistent: False
Issues: ['Owner name mismatch between EC and Deed', 'Property address mismatch between EC and Deed']
Response:
 property_id='70/I' owner_name='செல்வம் sl0 கணேசன்' address='Door No. 70/I, Kattuma Akinainagar, Kaniyambadi Panchayat' all_documents_consistent=False issues=['Owner name mismatch between EC and Deed', 'Property address mismatch between EC and Deed']


Enter JSON input or exit:  {   "file": "C:/Users/Raghul mani M/Downloads/COM_SALE_DEED.pdf",   "prompt": " analyse and extract information(extract property id, owners name, address) from current document sale deed  and all_documents_consistent(True/False) , issues('issues': ['Owner name mismatch between EC and Deed']) from both the current and previous documents"  }


Property ID: Dwelling Unit in AWHO, Raman Vihar Project, Anna Nagar, Chinnavedampatti Village (Number unknown)
Owner Name: Purchaser Name Not Provided in Document
Address: Flat No.... of. Apartments, AWHO, Raman Vihar Project, Anna Nagar, Chinnavedampatti Village, Coimbatore North Taluk, Coimbatore-641049
All Documents Consistent: False
Issues: ["Purchaser's name is missing in the Sale Deed (current document).", 'Specific Dwelling Unit Number for the property sold is missing in the Sale Deed (current document).', "Property identification discrepancy: The Sale Deed describes a 'Dwelling Unit in AWHO, Raman Vihar Project', while the Encumbrance Certificate (House Tax Receipt) refers to 'கதவு இலக்கம் அல்லது70/I' (Door No. 70/I). A direct match between these identifiers is not evident.", "Owner name inconsistency: The Encumbrance Certificate lists 'செல்வம் sl0 கணேசன்' (Selvam S/o Ganesan) as the tax-payer/owner for 'Door No. 70/I'. The Sale Deed's vendor is 'ARMY WELFARE HOUSING ORGANISATI

Enter JSON input or exit:  exit


Exiting...


Given multiple diagnostic text reports (e.g., blood test report text, ECG textual summary, X-ray observation text), generate a structured health summary and identify inconsistencies or anomalies between them.
You will simulate multiple document inputs as text strings,
expected output schema :
{
  "patient_name": "Anjali R",
  "age": 42,
  "summary": "High cholesterol with mild ECG irregularity. Overall moderate health risk.",
  "risk_level": "Moderate",
  "all_reports_consistent": true,
  "issues": ["Cholesterol above normal"],
  "follow_up_required": true
}

In [5]:
from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import List, Optional


class healthsummary(BaseModel):
    patient_name: str
    age: int
    summary: str
    risk_level: str
    all_reports_consistent: bool
    issues: List[str]
    follow_up_required: bool


client = genai.Client()


blood_test_report = "Blood test shows cholesterol level at 240 mg/dL which is high."
ecg_report = "ECG reveals mild irregular heartbeat with no severe episodes."
xray_report = "Chest X-ray indicates no abnormalities."


input_texts = [
    f"Blood Test Report: {blood_test_report}",
    f"ECG Report: {ecg_report}",
    f"X-ray Report: {xray_report}",
    "Generate a structured health summary including inconsistencies and risk level."
]

contents = [types.Content(role="user", parts=[types.Part(text=txt)]) for txt in input_texts]


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=HealthSummary
    )
)


health_summary = response.parsed


print("Patient Name:", health_summary.patient_name)
print("Age:", health_summary.age)
print("Summary:", health_summary.summary)
print("Risk Level:", health_summary.risk_level)
print("All Reports Consistent:", health_summary.all_reports_consistent)
print("Issues:", health_summary.issues)
print("Follow-up Required:", health_summary.follow_up_required)


NameError: name 'HealthSummary' is not defined

In [17]:
from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import List, Optional


class healthsummary(BaseModel):
    patient_name: str
    age: int
    summary: str
    risk_level: str
    all_reports_consistent: bool
    issues: List[str]
    follow_up_required: bool


client = genai.Client()


blood_test_report = "Blood test shows cholesterol level at 240 mg/dL which is high."
ecg_report = "ECG reveals mild irregular heartbeat with no severe episodes."
xray_report = "Chest X-ray indicates no abnormalities."


input_texts = [
    f"Blood Test Report: {blood_test_report}",
    f"ECG Report: {ecg_report}",
    f"X-ray Report: {xray_report}",
    "Generate a structured health summary including inconsistencies and risk level."
]

contents = [types.Content(role="user", parts=[types.Part(text=txt)]) for txt in input_texts]


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=healthsummary
    )
)


health_summary = response.parsed


print("Patient Name:", health_summary.patient_name)
print("Age:", health_summary.age)
print("Summary:", health_summary.summary)
print("Risk Level:", health_summary.risk_level)
print("All Reports Consistent:", health_summary.all_reports_consistent)
print("Issues:", health_summary.issues)
print("Follow-up Required:", health_summary.follow_up_required)


Patient Name: N/A
Age: 0
Summary: Blood test shows high cholesterol (240 mg/dL) and ECG reveals a mild irregular heartbeat. Chest X-ray indicates no abnormalities.
Risk Level: Moderate
All Reports Consistent: True
Issues: ['High cholesterol', 'Mild irregular heartbeat']
Follow-up Required: True
